In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.3.2`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = SparkSession.builder()
  .appName("SparkTest")
  .master("local[*]")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark is ready!")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark is ready!


import $ivy.$                                      

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@a01a5c2

In [2]:
import spark.implicits._
import java.sql.Timestamp
case class Signups(id: Int, time_stamp: Timestamp)
case class Confirmations(user_id: Int, time_stamp: Timestamp, action: String)

val data = Seq[(Int, Timestamp)]((3, Timestamp.valueOf("2020-03-21 10:16:13")), (7, Timestamp.valueOf("2020-01-04 13:57:59")), (2, Timestamp.valueOf("2020-07-29 23:09:44")), (6, Timestamp.valueOf("2020-12-09 10:39:37")))
val signups = data.toDF("id", "time_stamp")
    .as[Signups]
signups.show()

val data2 = Seq[(Int, Timestamp, String)]((3, Timestamp.valueOf("2021-01-06 03:30:46"), "timeout"), (3, Timestamp.valueOf("2021-07-14 14:00:00"), "timeout"), (7, Timestamp.valueOf("2021-06-12 11:57:29"), "confirmed"), (7, Timestamp.valueOf("2021-06-13 12:58:28"), "confirmed"), (7, Timestamp.valueOf("2021-06-14 13:59:27"), "confirmed"), (2, Timestamp.valueOf("2021-01-22 00:00:00"), "confirmed"), (2, Timestamp.valueOf("2021-02-28 23:59:59"), "timeout"))
val confirmations = data2.toDF("user_id", "time_stamp", "action")
    .as[Confirmations]
confirmations.show()

+---+-------------------+
| id|         time_stamp|
+---+-------------------+
|  3|2020-03-21 10:16:13|
|  7|2020-01-04 13:57:59|
|  2|2020-07-29 23:09:44|
|  6|2020-12-09 10:39:37|
+---+-------------------+

+-------+-------------------+---------+
|user_id|         time_stamp|   action|
+-------+-------------------+---------+
|      3|2021-01-06 03:30:46|  timeout|
|      3|2021-07-14 14:00:00|  timeout|
|      7|2021-06-12 11:57:29|confirmed|
|      7|2021-06-13 12:58:28|confirmed|
|      7|2021-06-14 13:59:27|confirmed|
|      2|2021-01-22 00:00:00|confirmed|
|      2|2021-02-28 23:59:59|  timeout|
+-------+-------------------+---------+



import spark.implicits._

import java.sql.Timestamp

defined class Signups
defined class Confirmations
data: Seq[(Int, Timestamp)] = List(
  (3, 2020-03-21 10:16:13.0),
  (7, 2020-01-04 13:57:59.0),
  (2, 2020-07-29 23:09:44.0),
  (6, 2020-12-09 10:39:37.0)
)
signups: Dataset[Signups] = [id: int, time_stamp: timestamp]
data2: Seq[(Int, Timestamp, String)] = List(
  (3, 2021-01-06 03:30:46.0, "timeout"),
  (3, 2021-07-14 14:00:00.0, "timeout"),
  (7, 2021-06-12 11:57:29.0, "confirmed"),
  (7, 2021-06-13 12:58:28.0, "confirmed"),
  (7, 2021-06-14 13:59:27.0, "confirmed"),
  (2, 2021-01-22 00:00:00.0, "confirmed"),
  (2, 2021-02-28 23:59:59.0, "timeout")
)
confirmations: Dataset[Confirmations] = [user_id: int, time_stamp: timestamp ... 1 more field]

The confirmation rate of a user is the number of 'confirmed' messages divided by the total number of requested confirmation messages. The confirmation rate of a user that did not request any confirmation messages is 0. Round the confirmation rate to two decimal places.

Write a solution to find the confirmation rate of each user.

Return the result table in any order.

The result format is in the following example.

| user_id | confirmation_rate |
|---------|-------------------|
| 6       | 0.00              |
| 3       | 0.00              |
| 7       | 1.00              |
| 2       | 0.50              |

In [3]:
val allCnt = confirmations.groupBy(confirmations("user_id"))
    .agg(count(confirmations("user_id")).as("count"))

val confirmedCnt = confirmations.filter(confirmations("action") === "confirmed")
    .groupBy(confirmations("user_id"))
    .agg(count(confirmations("user_id")).as("confirmed_count"))

val confirmationRate = allCnt.join(confirmedCnt, Seq("user_id"), "left_outer")
    .na.fill(0, Seq("confirmed_count"))
    .select(col("user_id"), round(col("confirmed_count")/col("count"), 2).as("confirmation_rate"))

allCnt.show()
confirmedCnt.show()
confirmationRate.show()

+-------+-----+
|user_id|count|
+-------+-----+
|      3|    2|
|      7|    3|
|      2|    2|
+-------+-----+

+-------+---------------+
|user_id|confirmed_count|
+-------+---------------+
|      7|              3|
|      2|              1|
+-------+---------------+

+-------+-----------------+
|user_id|confirmation_rate|
+-------+-----------------+
|      3|              0.0|
|      7|              1.0|
|      2|              0.5|
+-------+-----------------+



allCnt: DataFrame = [user_id: int, count: bigint]
confirmedCnt: DataFrame = [user_id: int, confirmed_count: bigint]
confirmationRate: DataFrame = [user_id: int, confirmation_rate: double]

In [4]:
signups.join(confirmationRate, (signups("id") === confirmationRate("user_id")), "left_outer")
    .na.fill(0, Seq("confirmation_rate"))
    .select(col("id").as("user_id"), col("confirmation_rate"))
    .show()

+-------+-----------------+
|user_id|confirmation_rate|
+-------+-----------------+
|      3|              0.0|
|      7|              1.0|
|      2|              0.5|
|      6|              0.0|
+-------+-----------------+



In [5]:
spark.stop()